In [1]:
import os
import re
import string
import boto3
from botocore.exceptions import ClientError
import awswrangler as wr
import pandas as pd
import numpy as np

SEED = 1234
N_SAMPLES = 100_000
np.random.seed(SEED)

import gensim.downloader as api
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.corpora import Dictionary
from gensim.matutils import corpus2dense, corpus2csc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from tqdm import tqdm

import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
chunks = pd.read_csv('../data/toxic_data.csv', chunksize=100000)
df = pd.concat(chunks)
df.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed.,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,0,2,0,0.373134,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental hospitals. Boorah,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,1,2,0,0.605263,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by not making this announcement himself.\n\nWhat an awful human being .....,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,2,3,7,0.666667,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\n\nIf you cop-suckers can't see a problem with this, then go suck the barrel of a Glock.",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,0,0,0,0.815789,0.065789,0.552632,0.592105,0.000000,0.684211,0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,0,1,0,0.550000,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [3]:
df['comment_text'] = df['comment_text'].fillna("")

In [4]:
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
for col in identity_columns + ['toxicity']:
    df.loc[:, col] = np.where(df[col] >= 0.5, True, False)

In [5]:
train_df = df[df['split'] == 'train']
test_df = df[df['split'] != 'train']

In [6]:
train_df.shape, test_df.shape

((1804875, 46), (194641, 46))

In [7]:
sample = df
# sample = train_df.sample(N_SAMPLES, random_state=SEED, ignore_index=True)
train_text, val_text, train_label, val_label = train_test_split(sample['comment_text'], sample['toxicity'], test_size=0.2, random_state=SEED)
# train_text, train_label = sample['comment_text'], sample['toxicity']
test_text, test_label = test_df['comment_text'], test_df['toxicity']

# train_label = train_label.astype('int').to_numpy()
# val_label = val_label.astype('int').to_numpy()
# test_label = test_label.astype('int').to_numpy()

In [8]:
train_text.shape, val_text.shape, test_text.shape

((1599612,), (399904,), (194641,))

In [9]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}


def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re


def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)

    def replace(match):
        return misspellings[match.group(0)]

    return misspellings_re.sub(replace, text)
    

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']


def clean_text(x):
    x = str(x)
    for punct in puncts + list(string.punctuation):
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    return re.sub('\d+', ' ', x)

In [10]:
# clean misspellings
train_text = train_text.apply(replace_typical_misspell)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(replace_typical_misspell)

# clean the text
train_text = train_text.apply(clean_text)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(clean_text)

# clean numbers
train_text = train_text.apply(clean_numbers)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(clean_numbers)

In [11]:
def simple_preproc(text):
    for line in text:
        yield simple_preprocess(line)

In [12]:
# train_lists = list(map(lambda x: simple_preprocess(x), train_text))
# test_lists = list(map(lambda x: simple_preprocess(x), test_text))  

In [13]:
# train_lists = [simple_preprocess(doc) for doc in tqdm(train_text)]
# # val_lists = [simple_preprocess(doc) for doc in val_text]
# test_lists = [simple_preprocess(doc) for doc in tqdm(test_text)]

In [14]:
# Create the Dictionary and Corpus
dictionary = corpora.Dictionary()

#allow_update=True - add new words to dictionary
bow_train = [dictionary.doc2bow(doc, allow_update=True) for doc in tqdm(simple_preproc(train_text))]
bow_val = [dictionary.doc2bow(doc, allow_update=False) for doc in tqdm(simple_preproc(val_text))]
bow_test = [dictionary.doc2bow(doc, allow_update=False) for doc in tqdm(simple_preproc(test_text))] 

dictionary.save("my_dictionary_full")
loaded_dict = corpora.Dictionary.load("my_dictionary_full")

1599612it [03:33, 7492.26it/s]
399904it [00:38, 10458.24it/s]
194641it [00:18, 10774.89it/s]


In [15]:
num_docs = dictionary.num_docs
num_terms = len(dictionary.keys())
print(f"Number of docs is {num_docs}, there are {num_terms} words in dictionary")

Number of docs is 1599612, there are 261552 words in dictionary


In [16]:
tfidf = models.TfidfModel(bow_train, dictionary=loaded_dict)
train_tfidf = tfidf[bow_train]
val_tfidf = tfidf[bow_val]
test_tfidf = tfidf[bow_test]

In [17]:
train_tfidf_sparse = corpus2csc(train_tfidf, num_terms=num_terms, num_docs=num_docs).T
val_tfidf_sparse = corpus2csc(val_tfidf, num_terms=num_terms).T
test_tfidf_sparse = corpus2csc(test_tfidf, num_terms=num_terms).T

In [18]:
# Getting all memory using os.popen()
total_memory, used_memory, free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])
  
# Memory usage
print("RAM memory % used:", round((used_memory/total_memory) * 100, 2))

RAM memory % used: 74.04


In [19]:
%%time
logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_tfidf_sparse, train_label)
oof_name = 'predicted_target'
test_df[oof_name] = logreg.predict_proba(test_tfidf_sparse)[:, 1]

CPU times: user 4min 29s, sys: 2min 26s, total: 6min 55s
Wall time: 2min 42s


In [20]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, oof_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[oof_name])

def compute_bpsn_auc(df, subgroup, label, oof_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[oof_name])

def compute_bnsp_auc(df, subgroup, label, oof_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[oof_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)
oof_name = 'predicted_target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, oof_name, 'toxicity')
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
6,black,1519,0.812314,0.785562,0.973499
2,homosexual_gay_or_lesbian,1065,0.814605,0.820933,0.962834
7,white,2452,0.835842,0.803607,0.974137
5,muslim,2040,0.840519,0.844202,0.963600
4,jewish,835,0.881251,0.881476,0.964172
8,psychiatric_or_mental_illness,511,0.902131,0.871045,0.970198
0,male,4386,0.913665,0.902575,0.966827
1,female,5155,0.918091,0.914620,0.963152
3,christian,4226,0.928295,0.939605,0.953229


In [21]:
def calculate_overall_auc(df, oof_name):
    true_labels = df['toxicity']
    predicted_labels = df[oof_name]
    return roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, oof_name))
print(f"FINAL SCORE IS {FINAL_SCORE}")

FINAL SCORE IS 0.9111456046087165


In [22]:
import pickle
# # Save to file in the current working directory
pkl_filename = "logreg_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(logreg, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    upload_pickle_model = pickle.load(file)


In [23]:
evaluation = [(train_tfidf_sparse, train_label), (val_tfidf_sparse, val_label)]

In [53]:
%%time
# fit model no training data
xgb_model = XGBClassifier(
    max_depth=6,
    eta=0.2,
    gamma=2.5,
    min_child_weight=4,
    subsample=0.5,
    reg_alpha=103.0, reg_lambda=0.913,
    verbosity=1,
    colsample_bytree=0.82,
    early_stopping_rounds=10, 
    objective='binary:logistic',
    )
xgb_model.fit(train_tfidf_sparse, train_label, eval_set=evaluation)

[0]	validation_0-logloss:0.56186	validation_1-logloss:0.56182
[1]	validation_0-logloss:0.47311	validation_1-logloss:0.47308
[2]	validation_0-logloss:0.40819	validation_1-logloss:0.40827
[3]	validation_0-logloss:0.36139	validation_1-logloss:0.36157
[4]	validation_0-logloss:0.32739	validation_1-logloss:0.32765
[5]	validation_0-logloss:0.30292	validation_1-logloss:0.30330
[6]	validation_0-logloss:0.28465	validation_1-logloss:0.28507
[7]	validation_0-logloss:0.27087	validation_1-logloss:0.27133
[8]	validation_0-logloss:0.25925	validation_1-logloss:0.25977
[9]	validation_0-logloss:0.25036	validation_1-logloss:0.25092
[10]	validation_0-logloss:0.24376	validation_1-logloss:0.24438
[11]	validation_0-logloss:0.23839	validation_1-logloss:0.23912
[12]	validation_0-logloss:0.23428	validation_1-logloss:0.23507
[13]	validation_0-logloss:0.23099	validation_1-logloss:0.23187
[14]	validation_0-logloss:0.22813	validation_1-logloss:0.22908
[15]	validation_0-logloss:0.22569	validation_1-logloss:0.22666
[1

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.82,
              early_stopping_rounds=10, enable_categorical=False, eta=0.2,
              eval_metric=None, gamma=2.5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.200000003, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=4,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=103.0, ...)

In [58]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.82,
              early_stopping_rounds=10, enable_categorical=False, eta=0.2,
              eval_metric=None, gamma=2.5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.200000003, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=4,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=103.0, ...)

In [59]:
xgb_model.save_model("model_xgb.json")

In [60]:
# xgb_model2 = xgb.XGBClassifier()
# xgb_model2.load_model("model_xgb.json")

In [61]:
oof_name = 'predicted_target_xgb'
test_df[oof_name] = xgb_model.predict_proba(test_tfidf_sparse)[:, 1]

ValueError: Length of values (1) does not match length of index (194641)

In [ ]:
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, oof_name, 'toxicity')
bias_metrics_df

In [ ]:
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, oof_name))
print(f"FINAL SCORE IS {FINAL_SCORE}")

In [44]:
test_strings = ["you've fucked up Fd asshole $5", "55 go fuck yourself, fuckin' fuck. Die!@", "I am lesbian and I proud of it",
               "BLM", "BLM sucks", "I think people like you should die", "I love you friends", "BLM $uck$",
               "I hate software development", "I hate software development and you", 
               "Go and kill this motherfucke"]

for text in test_strings:
    line = replace_typical_misspell(text)
    line = clean_text(line)
    line = clean_numbers(line)
    bow = [dictionary.doc2bow(doc, allow_update=False) for doc in simple_preproc([line])]
    test_tfidf = tfidf[bow]
    test_tfidf_sparse = corpus2csc(test_tfidf, num_terms=num_terms).T
    preds_lr = upload_pickle_model.predict_proba(test_tfidf_sparse)[:, 1]
    # preds_xgb = model.predict_proba(test_tfidf_sparse)[:, 1]
    print(f"Text: {text}, Logreg = {preds_lr[0]}")

Text: you've fucked up Fd asshole $5, Logreg = 0.9944592462545216
Text: 55 go fuck yourself, fuckin' fuck. Die!@, Logreg = 0.9983772480620811
Text: I am lesbian and I proud of it, Logreg = 0.23600877048268767
Text: BLM, Logreg = 0.04387685089123575
Text: BLM sucks, Logreg = 0.9864814460174348
Text: I think people like you should die, Logreg = 0.515300779442672
Text: I love you friends, Logreg = 0.013271473103748214
Text: BLM $uck$, Logreg = 0.06574767966618525
Text: I hate software development, Logreg = 0.01869637873081117
Text: I hate software development and you, Logreg = 0.0261994046957514
Text: Go and kill this motherfucke, Logreg = 0.9565406708712991


In [45]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 200,
        'seed': SEED
    }

In [46]:
scores = []
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']), min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']),
                    verbosity=1, objective='binary:logistic', eval_metric="auc", early_stopping_rounds=10)
    
    evaluation = [(train_tfidf_sparse, train_label), (val_tfidf_sparse, val_label)]
    
    clf.fit(train_tfidf_sparse,
            train_label,
            eval_set=evaluation,
            verbose=False)
    

    pred = clf.predict(val_tfidf_sparse)
    score = roc_auc_score(val_label, pred>0.7)
    scores.append(score)
    return {'loss': -score, 'status': STATUS_OK }

In [48]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

100%|██████████| 100/100 [17:08<00:00, 10.29s/trial, best loss: -0.5]


In [49]:
print(f"The best hyperparameters are : \n{best_hyperparams}")

The best hyperparameters are : 
{'colsample_bytree': 0.8261009556047648, 'gamma': 2.5362056282767274, 'max_depth': 6.0, 'min_child_weight': 4.0, 'reg_alpha': 103.0, 'reg_lambda': 0.9139711787883124}
